In [1]:
import requests
import pandas as pd
import datetime as dt
import subprocess
import json
import os
import re
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def grabNewDataAndStoreCSV(kind):
    keys = open('keys.txt')
    read_list = keys.readlines()
    for i in range(len(read_list)):
        text = read_list[i]
        read_list[i] = text.rstrip()
    user, token = read_list
    auth = user + ':' + token
    url = 'https://api.github.com/repos/movidius/ncappzoo/traffic/' + kind
    today_dt = dt.datetime.today()
    today = '_'.join(str(today_dt).split(' '))
    today = '-'.join(str(today.split('.')[0]).split(':'))
    folder_name = 'ncappzoo_' + kind + '_' + today
    file_name = folder_name + '/' + 'ncappzoo_' + kind + '_' + today

    os.mkdir(folder_name)
    test = subprocess.Popen(["curl", "-u", auth, url, ">", file_name], stdout=subprocess.PIPE, shell=True)
    out, err = test.communicate()
    decoded_string = out.decode("utf-8").replace("'", '""').replace('\n', '')
    data = json.loads(decoded_string)
    
    counts = data['count']
    uniques = data['uniques']
    timestamp_data = data[kind]
    df = pd.DataFrame(timestamp_data)
    
    timestamps = [x.split('T')[0] for x in df['timestamp'].values]
    df['day'] = [x.split('-')[2] for x in timestamps]
    df['mo'] = [x.split('-')[1] for x in timestamps]
    df['yr'] = [x.split('-')[0] for x in timestamps]
    df['collection_timestamp'] = [today_dt for x in range(len(timestamps))]
    
    json_string = json.dumps(data, indent=4, sort_keys=True)
    f = open(file_name + '.json', 'a')
    f.write(json_string)
    f.close()
    csv_string = df.to_csv(file_name + '.csv', index=False)
    return file_name + '.csv'

In [3]:
def construct_new_ref(old_file_path, new_file_path, kind):
    old_master_df = pd.read_csv(old_file_path)
    new_df = pd.read_csv(new_file_path)
    yrs = [x for x in new_df['yr'].values]
    mos = [x for x in new_df['mo'].values]
    dys = [x for x in new_df['day'].values]
    new_df['yr_mo_day'] = [dt.datetime(yrs[i], mos[i], dys[i]) for i in range(len(yrs))]
    
    # Construction of new ref
    new_master = old_master_df.append(new_df).groupby('timestamp').max().reset_index()
    yrs = [x for x in new_master['yr'].values]
    mos = [x for x in new_master['mo'].values]
    dys = [x for x in new_master['day'].values]
    new_master['yr_mo_day'] = [dt.datetime(yrs[i], mos[i], dys[i]) for i in range(len(yrs))]
    
    today_dt = dt.datetime.today()
    today = '_'.join(str(today_dt).split(' '))
    today = '-'.join(str(today.split('.')[0]).split(':'))
    file_name = 'temp/updated_' + kind + '_' + today
    csv_string = new_master.to_csv(file_name + '.csv', index=False)
    return file_name + '.csv'

In [46]:
# New
new_file_views = grabNewDataAndStoreCSV('views')
old_ref_views = 'ref_views_2019-10-26.csv'
new_ref_views = construct_new_ref(old_ref_views, new_file_views, 'views')

In [47]:
new_file_clones = grabNewDataAndStoreCSV('clones')
old_ref_clones = 'ref_'
new_ref_clones = construct_new_ref(old_ref_clones, new_file_clones, 'clones')

In [48]:
# Test
testpd_clones = pd.read_csv(new_ref_clones)
testpd2_clones = pd.read_csv('temp/updated_clones_2019-10-19_18-42-52.csv')

In [49]:
testpd_views = pd.read_csv(new_ref_views)
testpd2_views = pd.read_csv('temp/updated_views_2019-10-19_19-02-05.csv')

In [50]:
len(testpd_views), len(testpd2_views)

(41, 35)

In [51]:
len(testpd_clones), len(testpd2_clones)

(40, 34)